In [1]:
import Base.copy
using Distributions
include("Mondrian_Forest_Classifier.jl"); 
include("Mondrian_extention_utils.jl");
include("Mondrian_extention.jl")

Extend_Mondrian_Block! (generic function with 1 method)

In [2]:
function 
    FakedataClassif(n_obs,d)
    x=randn((n_obs,d))
    y=(sum(x*randn(d),2) .> mean(sum(x*randn(d),2)))
    y=y+1
    y=Int.(y)
    return x,y
end

FakedataClassif (generic function with 1 method)

In [3]:
data = FakedataClassif(50000,100)
d_new=FakedataClassif(1000,100)

([0.976269 -0.88885 … 0.21028 0.038478; 1.19144 0.23194 … 0.677823 0.354119; … ; 0.310332 -0.756372 … 0.469329 0.753217; 0.320461 0.236738 … 1.11474 0.214628], [1, 1, 2, 2, 2, 1, 2, 2, 2, 2  …  2, 1, 1, 2, 2, 2, 1, 1, 1, 2])

In [4]:
T = Mondrian_Tree()

Mondrian_Tree(Nullable{Mondrian_Node}(), Mondrian_Node[])

In [5]:
T_online = Sample_Mondrian_Tree!(T,1e9,data[1],data[2]);  #compute base tree

In [6]:
for i=1:length(d_new[2])
online_data=[d_new[1][i,:],d_new[2][i]]
T_online = Extend_Mondrian_Tree!(T_online,1e9,online_data);  #extent base tree    
end    


In [7]:
All_data = zeros(length(data[2])+length(d_new[2]),length(data[1][1,:]))
All_data[1:length(data[2]),:]=data[1]
All_data[length(data[2])+1:length(data[2])+length(d_new[2]),:]=d_new[1]

all_labels = Array{Int64}(length(data[2])+length(d_new[2]),1)
all_labels[1:length(data[2]),:]=data[2]
all_labels[length(data[2])+1:length(data[2])+length(d_new[2]),:]=d_new[2]

1000-element Array{Int64,1}:
 1
 1
 2
 2
 2
 1
 2
 2
 2
 2
 2
 2
 1
 ⋮
 2
 1
 2
 1
 1
 2
 2
 2
 1
 1
 1
 2

In [8]:
T_full = Sample_Mondrian_Tree!(T,1e9,All_data,all_labels);  #compute full tree of the whole dataset

### Compare prediction accuracy

In [9]:
compute_predictive_posterior_distribution!(T_full,10*size(All_data,2))

In [10]:
pred = []
    for i in 1:size(All_data,1)
        p = predict!(T_full,All_data[i,:],10*size(All_data,2))
        if p[1] > p[2]
            push!(pred,1)
        else
            push!(pred,2)
        end
    end


In [11]:
counter = 0
for i=1:1001
    if pred[i]==all_labels[i]
        counter = counter+1
    end
end
counter = counter /1001

0.5744255744255744

In [12]:
compute_predictive_posterior_distribution!(T_online,10*size(All_data,2))

In [13]:
 pred = []
        for i in 1:size(All_data,1)
            p = predict!(T_online,All_data[i,:],10*size(All_data,2))
            if p[1] > p[2]
                push!(pred,1)
            else
                push!(pred,2)
            end
        end

In [14]:
counter = 0
for i=1:1001
    if pred[i]==all_labels[i]
        counter = counter+1
    end
end
counter = counter /1001

0.5744255744255744

## Conclusion: get the same accuracy weather we incorperate the datapoint later or immediatly

## To Do:

Clean up utils files, test on more classes, test on extending to more datapoints successively, implement paused mondrians 